In [1]:
from unityagents import UnityEnvironment
import numpy as np
import os
import random
import torch
from collections import deque
import matplotlib.pyplot as plt
from ddpg_agent import Agent
import sys

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"

#env = UnityEnvironment(file_name='./Tennis_Linux/Tennis.x86_64')
no_graphics_flag=True
env = UnityEnvironment(file_name='./Tennis_Linux/Tennis.x86_64',no_graphics=no_graphics_flag)

# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]


# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


In [3]:
# number of agents 
num_agents = len(env_info.agents)
print('Number of agents:', num_agents)

# size of each action
action_size = brain.vector_action_space_size
print('Size of each action:', action_size)

# examine the state space 
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], state_size))
print('The state for the first agent looks like:', states[0])

print('The state for the second agent looks like:', states[0])

Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24
The state for the first agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]
The state for the second agent looks like: [ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -6.65278625 -1.5
 -0.          0.          6.83172083  6.         -0.          0.        ]


In [4]:
## define ddpg
def ddpg(my_agents,num_agents,n_episodes=2000, max_t=10000):

    scores_all = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    scores_mean = []
    max_t_totals = []
    max_t_window = deque(maxlen=100)
    mem = []

    for i_episode in range(1, n_episodes+1):

        env_info = env.reset(train_mode=True)[brain_name]
        my_agents.reset()

        states = env_info.vector_observations                  # get the current state (for each agent)
        actions = np.zeros((num_agents, action_size))          # initialize the action space (for each agent)
        scores = np.zeros(num_agents)                          # initialize the score (for each agent)
        max_t_lengths = []

        for t in range(max_t):

            # agent chooses action
            actions=my_agents.act(states)

            # interact with environment
            env_info = env.step(actions)[brain_name]
            (next_states,rewards,dones) = (
                env_info.vector_observations,
                env_info.rewards,
                env_info.local_done)

            # update agent
            #for state,action,reward,next_state,done in zip(states, actions, rewards, next_states, dones):
            #    my_agents.step(state,action,reward,next_state,done)
            for i in range(num_agents):
                my_agents.step(states[i],actions[i],rewards[i],next_states[i],dones[i])
            
            states = next_states
            scores += rewards

            if np.any(dones):
                break
                max_t_len.append(t)

        mean_score = np.max(scores)
        scores_window.append(mean_score)       # save most recent score
        scores_all.append(scores)              # save most recent score
        scores_mean.append(np.mean(scores_window))
        mem.append(len(my_agents.memory))
        
        max_t_window.append(t)
        max_t_totals.append(np.mean(max_t_window))

        #print('\rEpisode {}\tAverage Score: {:.4f}\tScores: {}\tMax t: {:.2f}\tmem: {}'
        #      .format(i_episode, np.mean(scores_window),scores,np.mean(max_t_window),len(my_agents.memory)), end="")
        print('\rEpisode {}\tAverage Score: {:.4f}\tMax t: {:.2f}\tmem: {}'
              .format(i_episode, np.mean(scores_window),np.mean(max_t_window),len(my_agents.memory)), end="")
        if i_episode % 50 == 0:
            print('\rEpisode {}\tAverage Score: {:.4f}'.format(i_episode, np.mean(scores_window)))

        if np.mean(scores_window)>=0.5:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.4f}'.format(i_episode-100, np.mean(scores_window)))

            torch.save(my_agents.actor_local.state_dict(), 'checkpoint_my_actor.pth')
            torch.save(my_agents.critic_local.state_dict(), 'checkpoint_my_critic.pth')
            break

    max_result = max(scores_mean)
    last_result = scores_mean[-1]


    with open('results.txt','a') as fh:
        
        fh.write("RESULTS (max,last,scores) {} {} {}\n".format(
            str(max_result),
            str(last_result),
            str(scores_mean)
        ))
        fh.write("LAST RESULT: {}\n".format(str(last_result)))
        fh.write("MAX RESULT: {}\n\n".format(str(max_result)))


    return (scores_all,scores_mean,max_t_totals)


In [5]:
def run_trial(config):
    
    ## setup my agent
    agent_config = config['agent_config']
    my_agent = Agent(**agent_config)

    ## run the agent
    (all_scores,mean_scores,max_t_lengths) = ddpg(
        my_agent,
        **config['ddpg_config']
    )
    

    ## graph scores here
    plt.figure(1)
    plt.subplot(211)
    #plt.plot(all_scores)
    plt.plot(mean_scores,label='mean')
    #plt.legend()
    plt.ylabel('Score')
    plt.xlabel('Episode #');
    
    plt.subplot(212)
    plt.plot(max_t_lengths)
    plt.ylabel('Length')
    plt.xlabel('Episode #');
    
    plt.show()
    
    return all_scores

In [ ]:
cfg = {
        'ddpg_config':{ 
            'num_agents':num_agents,
            'n_episodes':1800,
            'max_t':1000,
        },
        'agent_config':{
            'state_size':state_size,
            'action_size':action_size,
            'random_seed':123,
            'buffer_size':100000,   # replay buffer size
            'batch_size':64,       # minibatch size
            'lr_actor':.001, 
            'lr_critic':.0001,
            'gamma':0.99,
            'tau':.003,
            'weight_decay':0.0,
            'ou_theta':0.15,
            'ou_sigma':0.2, # 0.2
            'actor_nn':[256,256],
            'critic_nn':[256,128,64],
            'epsilon':1.0,
            'epsilon_decay':0.00001, 
            'update_every':18, 
            'num_updates':10,
            'hard_copy':1,
            'prefill':0,
            
        }
    }

with open('results.txt','a') as fh:
    fh.write(str(cfg))
    fh.write("\n")
    
all_scores=run_trial(cfg)

Episode 50	Average Score: 0.0000	Max t: 13.20	mem: 1420
Episode 100	Average Score: 0.0057	Max t: 14.14	mem: 3028
Episode 150	Average Score: 0.0094	Max t: 14.81	mem: 4582
Episode 200	Average Score: 0.0037	Max t: 13.98	mem: 6024
Episode 250	Average Score: 0.0000	Max t: 13.31	mem: 7444
Episode 300	Average Score: 0.0010	Max t: 13.39	mem: 8902
Episode 350	Average Score: 0.0010	Max t: 13.42	mem: 10328
Episode 400	Average Score: 0.0161	Max t: 16.05	mem: 12312
Episode 450	Average Score: 0.0303	Max t: 18.70	mem: 14268
Episode 500	Average Score: 0.0256	Max t: 18.13	mem: 16138
Episode 550	Average Score: 0.0210	Max t: 17.49	mem: 17966
Episode 600	Average Score: 0.0199	Max t: 17.62	mem: 19862
Episode 650	Average Score: 0.0162	Max t: 17.34	mem: 21634
Episode 700	Average Score: 0.0198	Max t: 18.86	mem: 23834
Episode 750	Average Score: 0.0555	Max t: 28.16	mem: 27466
Episode 800	Average Score: 0.0732	Max t: 33.56	mem: 30746
Episode 850	Average Score: 0.0673	Max t: 32.23	mem: 34112
Episode 900	Average S

In [ ]:
## close up shop
env.close()